In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt



In [3]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    #transforms.Resize((128, 128)),  # Resize images to 128x128 pixels
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

cifar10_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)


Files already downloaded and verified


In [4]:
# Define the U-Net architecture
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Define encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Define decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
unet_model = UNet(in_channels=3, out_channels=3)
criterion = nn.MSELoss()
optimizer = optim.Adam(unet_model.parameters(), lr=0.001)

In [5]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    num_batches = 0
    total_loss = 0
    for i, data in enumerate(train_loader, 0):
        inputs, _ = data

        optimizer.zero_grad()
        outputs = unet_model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        total_loss = loss.item()
        num_batches+=1


    avg_epoch_loss = total_loss / num_batches
    print(f'Epochs {epoch}:')
    print(f'        Avg Epoch Loss: {avg_epoch_loss}')




/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 3, 128, 128])) that is different to the input size (torch.Size([64, 3, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (64) must match the size of tensor b (128) at non-singleton dimension 3

In [ ]:
# Define diffusion model using the trained U-Net model
class DiffusionModel:
    def __init__(self, unet_model, noise_level):
        self.unet_model = unet_model
        self.noise_level = noise_level

    #adds gaussian noise to image
    def add_noise(self, image, noise_level):

        noise = np.random.normal(loc=0.0, scale=noise_level, size=image.shape)
        image_noised  = image + noise
        image_noised = np.clip(image_noised, 0, 1)

        return image_noised

    #one iteration of diffusion
    def diffusion_step(self, image):

        image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0)
        updated_image_tensor = self.unet_model(image_tensor)
        updated_image = updated_image_tensor.squeeze(0).detach().numpy()
        updated_image = self.add_noise(updated_image, self.noise_level)

        return updated_image




In [ ]:
noise_level = 0.2

diffusion_model = DiffusionModel(unet_model, noise_level)
generated_image = np.random.rand(3, 32, 32)
for i in range(5):
    generated_image = diffusion_model.diffusion_step(generated_image)
    plt.imshow(generated_image.transpose(1, 2, 0))
    plt.axis('off')
    plt.show()